In [12]:
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

import gurobipy as grb
from gurobipy import GRB

# Input Problem Parameters

## Import du problème

In [13]:
from utils.extract_parameters import extract_parameters
from utils.define_nums import define_nums
from utils.domination_3D import liste_dominants, graph_dominant

In [14]:
################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "large"

## To change according to preference system
input_projet_max = 1
input_longueur_max = 35

In [15]:
################################
######## DO NOT CHANGE #########
################################
map_employe_indice, map_job_indice, map_qualif_indice, conge, qualif, d, p, b, besoin = extract_parameters(taille)
nombre_employes, horizon, nombre_qualifs, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

## Définissons les paramètres

In [16]:
## u_j, le projet j est réalisé
u_shape = nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x_shape = [(i,j,k, n) for i in range(nombre_qualifs) for j in range(nombre_projets) for k in range(nombre_employes) for n in range(horizon)]

## t_k_j, l'employé k travaille sur le projet j
t_shape = [(i,j) for i in range(nombre_employes) for j in range(nombre_projets)]

## begin_j, la date de début effective
begin_shape = nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end_shape = nombre_projets

## delay_j, le nombre de jours de retard par projet j
delay_shape = nombre_projets

## longueur_j, la longueur du projet j
longueur_shape = nombre_projets

## projet_par_employe_k, le nombre de projet par employe
projet_par_employe_shape = nombre_employes

# majorant pour la contrainte de couverture
maj = max([besoin[i][j] for j in range(nombre_projets) for i in range(nombre_qualifs)]) + 1

# si le projet j est travaillé au jour n
job_in_process_shape = [(j, n) for j in range(nombre_projets) for n in range(horizon)]

# Model deployment

## Add the variables

In [17]:
model = grb.Model("Planning")

# L'affectation du planning, la variable clé
x = model.addVars(x_shape, vtype=GRB.BINARY, name="x")

## Les dates de fin des projets
dateFin = model.addVars(end_shape, lb = 0, ub = horizon, name = "dateFin")

## Les dates de début des projets
dateDebut = model.addVars(begin_shape, lb = 0, ub = horizon, name = "dateDebut")

## Le nombre de jours de delai
delay = model.addVars(delay_shape, lb = 0, ub = horizon, name = "delay")

## Les longueurs des projets
longueur = model.addVars(longueur_shape, lb = 0, ub = horizon, name = "longueur")

## Nombre de projet par employé
projet_par_employe = model.addVars(projet_par_employe_shape, lb = 0, ub = nombre_projets, name = "projet_par_employe")

## L'affectation sur un projet d'un employé
affectation = model.addVars(t_shape, vtype = GRB.BINARY, name = "affectation")

## La réalisation ou non d'un projet
u = model.addVars(u_shape, vtype = GRB.BINARY, name="u")

## Verification du projet
job_in_process = model.addVars(job_in_process_shape, vtype = GRB.BINARY, name = "job_in_process")

In [18]:
# Update le modele pour confirmer la création des variables
model.update()

## Add constraints

In [19]:
for i in range(nombre_qualifs):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                ## La qualification du personnel
                model.addConstr( x[i,j,k,n] <= qualif[i][k], name="QualificationPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                model.addConstr( x[i,j,k,n] <= (besoin[i][j] >=1), name = "BesoinQualifPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                ## Contraintes des congés
                model.addConstr( x[i,j,k,n] <= conge[k][n], name=f"CongesPersonnel[{i}{j}{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) == besoin[i][j]*u[j], name=f"UniciteRealisation[{i}{j}]")

## Unicité de l'affectation quotidienne
for k in range(nombre_employes):
    for n in range(horizon):
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for j in range(nombre_projets)) for i in range(nombre_qualifs)) <= 1, name=f"UniciteAffectation[{k}{n}]")
       
for j in range(nombre_projets):
    for n in range(horizon):
        model.addConstrs(((job_in_process[j,n] >= x[i,j,k,n]) for i in range(nombre_qualifs) for k in range(nombre_employes)), name=f"ContrainteJobInProcess[{j}{n}]")
    # start date
    model.addConstrs(((dateDebut[j] <= job_in_process[j,n]*n) for n in range(horizon)), name = f"ContrainteDateDebut[{j}]")

    # end date
    model.addConstrs(((job_in_process[j,n]*n <= dateFin[j]) for n in range(horizon)), name=f"FinProjet[{j}]")

    ## longueur
    model.addConstr(longueur[j] == dateFin[j] - dateDebut[j], name=f"LongueurProjet[{j}]")

    ## delay
    model.addConstr(delay[j] >= dateFin[j] - (d[j] - 1), name=f"DelaiProjet_difference[{j}]")    

## projet_par_employe
for k in range(nombre_employes):
    model.addConstr(projet_par_employe[k] == grb.quicksum( affectation[k,j] for j in range(nombre_projets)), name=f"FinProjet[{j}]")
    
## Contrainte de l'affectation à un projet selon les jours de travail
for k in range(nombre_employes):
    for j in range(nombre_projets):
        model.addConstr(affectation[k,j] <= grb.quicksum(grb.quicksum(x[i,j,k,n] for i in range(nombre_qualifs)) for n in range(horizon)), name=f"AffectationProjetQuicksum[{k}{j}]")    

## Fonctions d'export

In [20]:
# Edt des employés et le détail de la réalisation de chaque tâche (de façon ordonnée)
def create_schedule(model):
    schedules = []
    for i in range(nombre_qualifs):
        for j in range(nombre_projets):
            for k in range(nombre_employes):
                for n in range(horizon):
                    v = model.getVarByName(f"x[{i},{j},{k},{n}]")
                    if int(v.x) == 1:
                        schedules.append((map_employe_indice[k], n+1, map_job_indice[j], map_qualif_indice[i]))
    schedules.sort()
    return(schedules)

In [21]:
## Input i: input_longueur_max
def create_df(model,i):
    schedules = create_schedule(model)
    df = pd.DataFrame(schedules, columns=['Nom', 'Jour', 'Projet', 'Qualification'])
    df['Affectation'] = df[['Projet', 'Qualification']].values.tolist()

    df = df.pivot(index=['Nom'], 
                        columns=['Jour'],
                        values=['Affectation'])

    for k in range(nombre_employes):
        for n in range(horizon):
            name = map_employe_indice[k]
            if not conge[k][n]:
                try:
                    df.loc[(df.index==name),("Affectation",n+1)] = 'En congé'
                except:
                    continue

    df.fillna("Non affecté.e", inplace=True)
    df.to_excel("results/final/"+taille+"/"+str(i)+".xlsx")

## Fonctions objectif et optimisation

In [22]:
## bénéfice total
benefice_tot = grb.quicksum( (b[j]*u[j] - delay[j]*p[j]) for j in range(nombre_projets) )

# Nombre de projet maximum par personne
projet_max = model.addVar(lb= 0, ub=nombre_projets, name="projet_max")
model.addConstrs(projet_max>=projet_par_employe[k] for k in range(nombre_employes))
model.addConstr(projet_max==input_projet_max)

# Compacité des projets
longueur_max = model.addVar(lb = 0, ub=horizon, name="longueur_max")
model.addConstrs(longueur_max>=longueur[j] for j in range(nombre_projets))
model.addConstr(longueur_max==input_longueur_max)

model.ModelSense = GRB.MAXIMIZE
model.setObjective(benefice_tot)

model.update()

model.optimize()

create_df(model, input_longueur_max)
print(model.objVal)

model.reset()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 67052 rows, 16987 columns and 133596 nonzeros
Model fingerprint: 0xa4573b2c
Variable types: 67 continuous, 16920 integer (16920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 7e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 66576 rows and 15576 columns
Presolve time: 0.06s
Presolved: 476 rows, 1411 columns, 3458 nonzeros
Variable types: 0 continuous, 1411 integer (1397 binary)

Root relaxation: objective 4.285904e+02, 1138 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

<ipython-input-21-42613d47759d>:21: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df.to_excel("results/final/"+taille+"/"+str(i)+".xlsx")
